# Testing the implementation

In [1]:
import os
import gzip
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import tensorflow_datasets as tfds

from tuner import *
from models import *
from trainer import *
from preprocess import *
from callbacks import *

## 0 Data

Obtain the full data

In [2]:
parent_path = str(pathlib.Path(os.getcwd()).parent)
df = pd.read_csv(os.path.join(parent_path, 'data/data.csv'))
df = df.drop(['Unnamed: 0'], axis=1)
df.sample(5)

,optionid,securityid,strike,callput,date_traded,contract_price,market_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility
30623,150542330.0,506534.0,5.425,C,2007-07-02,0.0890,0.08950,5.498400,164.0,4.0,1.013530,0.053877,0.112082
42849,150384996.0,702263.0,12.400,C,2007-08-24,0.3250,0.37375,12.007600,472.0,63.0,0.968355,0.054433,0.301770
100504,165244417.0,702263.0,18.050,C,2020-02-26,0.1425,0.15750,17.808000,4.0,9.0,0.986593,0.016077,0.208561
13162,150247574.0,504880.0,40.000,C,2006-09-27,0.3930,0.39700,38.961799,7544.0,51.0,0.974045,0.054138,0.117770
40489,162271087.0,702263.0,16.200,C,2018-07-24,0.2250,0.21500,15.987000,20.0,59.0,0.986852,0.022610,0.137587


Create the first neueral network that uses the same set of inputs as the *Black-Scholes model*

In [3]:
dataframe_BS = np.vstack((df['strike'].values,
                      df['underlyings_price'].values,
                      df['days_to_maturity'].values,
                      df['volatility'].values,
                      df['rate'].values,
                      df['contract_price'].values)).T                

In [4]:
train_ds, valid_ds, test_ds = pipeline1(dataframe_BS, scaling=False)

train_copy, valid_copy, test_copy = pipeline1(dataframe_BS, prefetch=False)

(85999, 6) (10750, 6) (10750, 6)
(85999, 6) (10750, 6) (10750, 6)


We can use the `StandardScaler` to normalize the data or the `MinMaxScaler` to scale the data, but note that the latter approach will turn the `days_to_maturity` times to zero for the options with `2` days, which is actually a lot.

In [10]:
df['days_to_maturity'].value_counts().to_dict()[2.0]

2667

In [ ]:
normal_train_ds, normal_valid_ds, normal_test_ds = pipeline2(df, True, 'normalize', 32, 1000)
scaled_train_ds, scaled_valid_ds, scaled_test_ds = pipeline2(df, True, 'minmax', 32, 1000)

Inspect the elements in the datasets

In [11]:
normal_test_ds.element_spec

(TensorSpec(shape=(None, 5), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.float32, name=None))

## 1 Start training

Define some parameters

In [ ]:
print_num_epochs = 5 # print progress every print_num_epochs epochs

path_to_save = os.path.join(parent_path, 'NeuralNetwork/models/roughwork')  # path to save the model

patience = 10  

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

loss = tf.keras.losses.MeanAbsoluteError(name='loss')

metrics = tf.keras.metrics.MeanAbsolutePercentageError(name='accuracy')

num_epochs = 10  

input_shape = (5,)

num_layers = 3

hidden_units = [14, 14, 14]

output_shape = (1, )

batchnorm = True  

dropout = None

model = getModel(input_shape = input_shape,
                num_layers   = num_layers,
                 hidden_units = hidden_units,
                 output_shape = output_shape,
                 batchnorm = batchnorm,
                 dropout = dropout)
model.summary()

Define some callbacks  

In [ ]:
ckpt = CheckpointCallback(path_to_save)
printing =PrintProgress(num_epochs=print_num_epochs)
early_stop = tf.keras.callbacks.EarlyStopping(patience=patience, monitor='val_loss')
callbacks = [ckpt, printing, early_stop]

In [ ]:
testing = False
if testing:
    history = compile_and_fit(model,
                          optimizer,
                          loss,
                          num_epochs,
                          train_ds,
                          valid_ds,
                          metrics,
                          callbacks,
                          verbose=True
                          )

# 1.1 Tuning the hyperparameters

To tune the hyperparameter, we can use `keras_tuner`, which provides `RandomSearch`, `Hyperband`, and `BayesianSearch` methods for searching the best hyperparameters.  

To get started, we only tune the number of layers and the hidden units in each layer first, then we tune with learning rate together.

In [ ]:
import keras_tuner

We initialize the `keras_tuner` object for tuning hyperparameters; currently, there are two hyper models implemented, `tuneLayer` and `tuneLR`

In [ ]:
# random_tuner = keras_tuner.RandomSearch(
#     hypermodel=tunedModel, # the hypermodel to tune # can be tuneLR or tuneLayer
#     objective="val_loss", # the objective to optimize
#     max_trials=3, # the maximum number of trials to run
#     executions_per_trial=2, # the number of models generated on each trial
#     overwrite=True, # whether to overwrite previous trials
#     directory="hyperparams/RandomSearch", # the directory to save the trials
#     project_name="1", # the name of the project
# )  

# # get a summary of the range of hyperparameters to tune


random_tuner = customTuner(input_shape, output_shape, 
                            objective='val_loss', 
                            max_trials=10,
                            executions_per_trial=1,
                            overwrite=True,
                            directory='hyperparams/RandomSearch',
                            project_name='Black-Scholes')
random_tuner.search_space_summary()

In [ ]:
# start the search
random_tuner.search(train_ds, valid_ds, epochs=5)

Get the best compiled models

In [ ]:
hypermodel = customHyperModel()

best_model = hypermodel.build(random_tuner.get_best_hyperparameters()[0])

best_model.summary()

We can view the results of the search and use the hyperparameters displayed to train the model.

In [ ]:
random_tuner.results_summary()

Now train the best model obtained

In [ ]:
best_model.fit(train_ds, epochs = 10, validation_data = valid_ds)

Save the trained model

In [ ]:
path_to_model = os.path.join(parent_path, 'NeuralNetwork/models/BS-0909')

In [ ]:
best_model.save(path_to_model)

To load the model, use `tf.keras.models.load_model`

In [ ]:
from tensorflow import keras
model = keras.models.load_model(path_to_model)
model.evaluate(test_ds)

## Some other Tuners to try out  

But this is not the goal of the project right now.

We also try `Hyperband` which generates a large number of models and discard them in a tournament style.

In [ ]:
band_tuner = keras_tuner.Hyperband(
    hypermodel=tuneLR,
    objective="val_loss",
    max_epochs=10,
    factor=3,
    directory="hyperparams/Hyperband",
    project_name="1"
)
band_tuner.search_space_summary()
band_tuner.search(train_ds, epochs = 2, validation_data = valid_ds)  

In [ ]:
band_tuner.results_summary()

Finally, we try `BayesianOptimization`.

In [ ]:
bayes_tuner = keras_tuner.BayesianOptimization(
    hypermodel=tuneLR,
    objective="val_loss",
    max_trials=3,
    directory="hyperparams/BayesianOptimization",
    project_name="1"
)
bayes_tuner.search_space_summary()
bayes_tuner.search(train_ds, epochs = 2, validation_data = valid_ds) 

In [ ]:
bayes_tuner.results_summary()